In [2]:
import requests
from dotenv import load_dotenv
import os
import geojson
import json
import urllib
from shapely.geometry import shape
import simplejson


In [3]:
load_dotenv('.env')

True

In [4]:
with open("data/fao.geojson", "r") as pfile:
    polys = geojson.load(pfile)

In [5]:
def coords_to_string(poly):
    geom = shape(poly.geometry)
    geom = geom.simplify(0.000001, preserve_topology=True)
    return geom.wkt


In [6]:
def auth():
  headers = {
      'Content-Type': 'application/x-www-form-urlencoded',
  }

  data = [
    ('apikey', os.getenv("API_KEY")),
    ('grant_type', 'api_key'),
    ('client_id', 'IDP'),
  ]

  response = requests.post('https://authenticate.foundation.api.oneatlas.airbus.com/auth/realms/IDP/protocol/openid-connect/token', headers=headers, data=data)

  if response.ok:
      auth_token = response.json().get("access_token")
  
  return auth_token

In [7]:
def search(poly):
    url = "https://search.foundation.api.oneatlas.airbus.com/api/v1/opensearch"

    querystring = {
        "relation": "intersects", 
        "geometry": coords_to_string(poly),
        "acquisitionDate": "[2018-01-01T00:00:00.000Z,2018-12-31T00:00:00.000Z]"
        }

    headers = {
        'Cache-Control': "no-cache",
        'Authorization': f"Bearer {auth_token}"
        }

    response = requests.request("GET", url, params=querystring, headers=headers)

    if response.ok:
        json_data = json.loads(response.text)

        return json_data

In [8]:
json_data["features"][0]

NameError: name 'json_data' is not defined

In [7]:
polys[1]["geometry"]

{"coordinates": [[[[106.444089, 63.438518], [106.444125, 63.438538], [106.444161, 63.438557], [106.444196, 63.438577], [106.444232, 63.438597], [106.444268, 63.438617], [106.444304, 63.438637], [106.44434, 63.438657], [106.444375, 63.438677], [106.444411, 63.438697], [106.444447, 63.438717], [106.444483, 63.438737], [106.444519, 63.438757], [106.444554, 63.438777], [106.44459, 63.438797], [106.444626, 63.438817], [106.444662, 63.438837], [106.444698, 63.438856], [106.444733, 63.438876], [106.444769, 63.438896], [106.444805, 63.438916], [106.444841, 63.438936], [106.444877, 63.438956], [106.444913, 63.438976], [106.444948, 63.438996], [106.444984, 63.439016], [106.44502, 63.439036], [106.445056, 63.439056], [106.445091, 63.439076], [106.445127, 63.439096], [106.445163, 63.439116], [106.445199, 63.439135], [106.445235, 63.439155], [106.445271, 63.439175], [106.445306, 63.439195], [106.445342, 63.439215], [106.445378, 63.439235], [106.445414, 63.439255], [106.44545, 63.439275], [106.44548

In [9]:
def check_price(id, poly, product_type = "pansharpened", radiometric = "DISPLAY", format = "image/jp2"):
    url = "https://data.api.oneatlas.airbus.com/api/v1/prices"

    payload= f"""{{\"kind\": \"order.data.gb.product\",  
        \"products\": [{{            
                \"crsCode\": \"urn:ogc:def:crs:EPSG::4326\",            
                \"productType\": \"{product_type}\",            
                \"radiometricProcessing\": \"{radiometric}\",            
                \"aoi\": {{                
                    \"type\": \"{poly["geometry"]["type"]}\",                
                    \"coordinates\": {poly["geometry"]["coordinates"]}}},            
        \"id\": \"{id}\",            
        \"imageFormat\": \"{format}\"        
        }}]}}"""

    headers = {
        'Authorization': f"Bearer {auth_token}",
        'Content-Type': "application/json"
        }

    response = requests.request("POST", url, data=payload, headers=headers)

    print(response.text)

In [10]:
def order(id, poly, product_type = "pansharpened", radiometric = "DISPLAY", format = "image/jp2"):
    url = "https://data.api.oneatlas.airbus.com/api/v1/orders"

    payload= f"""{{\"kind\": \"order.data.gb.product\",  
        \"products\": [{{            
                \"crsCode\": \"urn:ogc:def:crs:EPSG::4326\",            
                \"productType\": \"{product_type}\",            
                \"radiometricProcessing\": \"{radiometric}\",            
                \"aoi\": {{                
                    \"type\": \"{poly["geometry"]["type"]}\",                
                    \"coordinates\": {poly["geometry"]["coordinates"]}}},            
        \"id\": \"{id}\",            
        \"imageFormat\": \"{format}\"       
        }}]}}"""

    headers = {
        'Authorization': f"Bearer {auth_token}",
        'Content-Type': "application/json"
        }

    response = requests.request("POST", url, data=payload, headers=headers)

    print(response.text)

In [11]:
def check_orders():
    url = "https://data.api.oneatlas.airbus.com/api/v1/orders"

    headers = {
        'Cache-Control': "no-cache",
        'Content-Type': "application/json",
        'Authorization': f"Bearer {auth_token}"
        }

    querystring = {
        "status": "delivered"
        }

    response = requests.request("GET", url, headers=headers)

    if response.ok:
        json_data = json.loads(response.text)

        return json_data

In [12]:
def retrieve(order_id):
    url = f"https://data.api.oneatlas.airbus.com/api/v1/contracts/{order_id}/orders"

    headers = {
        'Content-Type': "application/json",
        'Authorization': f"Bearer {auth_token}",
        'Cache-Control': "no-cache",
        }

    response = requests.request("GET", url, headers=headers)

    if response.ok:
        json_data = json.loads(response.text)

        return json_data


In [70]:
def download(link, filename):
    url = link

    headers = {
        'Content-Type': "application/json",
        'Authorization': f"Bearer {auth_token}",
        }

    with open(filename, "wb") as fl:
        response = requests.request("GET", url, headers=headers, allow_redirects=True)
        print(response.status_code)

        if response.ok:
            fl.write(response.content)

In [14]:
auth_token = auth()

In [15]:
data = search(polys[5])

In [33]:
data

pi/v1/items/7191f159-1da0-435a-80cf-0218c4ab98f6/images/a4dbddf0-6968-48a7-878d-7d39c0ce24a5/wms',
     'resourceId': 'a4dbddf0-6968-48a7-878d-7d39c0ce24a5',
     'type': 'WMS'},
    'imagesWmts': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/7191f159-1da0-435a-80cf-0218c4ab98f6/images/a4dbddf0-6968-48a7-878d-7d39c0ce24a5/wmts',
     'resourceId': 'a4dbddf0-6968-48a7-878d-7d39c0ce24a5',
     'type': 'WMTS'},
    'quicklook': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/7191f159-1da0-435a-80cf-0218c4ab98f6/quicklook',
     'name': 'QuickLook',
     'type': 'HTTP'},
    'thumbnail': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/7191f159-1da0-435a-80cf-0218c4ab98f6/thumbnail',
     'name': 'Thumbnail',
     'type': 'HTTP'},
    'wms': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/7191f159-1da0-435a-80cf-0218c4ab98f6/wms',
     'name': 'WMS',
     'type': 'WMS'},
    'wmts': {'href'

In [34]:
img_id = "193868db-9e2d-4bcb-a167-8bab1be09420"

In [23]:
links

{'self': {'href': 'https://data.api.oneatlas.airbus.com/api/v1/deliveries/3c6ba88c-9fa2-4e15-9a19-75fb26f0c4c0'},
 'contract': {'href': 'https://data.api.oneatlas.airbus.com/api/v1/contracts/cd55f842-5552-432b-b833-d29d00ca1643'},
 'catalogUrl': {'href': 'https://search.foundation.api.oneatlas.airbus.com/api/v1/opensearch?id=0f57fe19-46ad-4cbc-b0bc-64329bead6a0'},
 'catalogItem': {'href': 'https://search.foundation.api.oneatlas.airbus.com/api/v1/opensearch?id=0f57fe19-46ad-4cbc-b0bc-64329bead6a0'},
 'download': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/0f57fe19-46ad-4cbc-b0bc-64329bead6a0/download'},
 'payment': {'href': 'https://data.api.oneatlas.airbus.com/api/v1/contracts/cd55f842-5552-432b-b833-d29d00ca1643/payments/12687a62-a5fb-48ba-b1f9-dfa68d5b6d0d'}}

In [63]:
check_price(img_id, polys[5], product_type="bundle", radiometric="DISPLAY", format="image/geotiff")

59656, 60.703813], [100.59653, 60.703832], [100.596499, 60.703851], [100.596469, 60.70387], [100.596438, 60.703889], [100.596408, 60.703907], [100.596377, 60.703926], [100.596346, 60.703945], [100.596316, 60.703964], [100.596285, 60.703983], [100.596255, 60.704002], [100.596224, 60.704021], [100.596194, 60.70404], [100.596163, 60.704059], [100.596133, 60.704077], [100.596102, 60.704096], [100.596072, 60.704115], [100.596041, 60.704134], [100.596011, 60.704153], [100.59598, 60.704172], [100.595949, 60.704191], [100.595919, 60.70421], [100.595888, 60.704229], [100.595858, 60.704247], [100.595827, 60.704266], [100.595797, 60.704285], [100.595766, 60.704304], [100.595736, 60.704323], [100.595705, 60.704342], [100.595675, 60.704361], [100.595644, 60.70438], [100.595614, 60.704399], [100.595583, 60.704417], [100.595553, 60.704436], [100.595522, 60.704455], [100.595491, 60.704474], [100.595461, 60.704493], [100.59543, 60.704512], [100.5954, 60.704531], [100.595369, 60.70455], [100.595339, 60.

In [65]:
order(img_id, polys[5], product_type="
", radiometric="DISPLAY", format="image/geotiff")

 [100.5954, 60.704531], [100.595369, 60.70455], [100.595339, 60.704569], [100.595308, 60.704587], [100.595278, 60.704606], [100.595247, 60.704625], [100.595216, 60.704644], [100.595186, 60.704663], [100.595155, 60.704682], [100.595125, 60.704701], [100.595094, 60.70472], [100.595064, 60.704739], [100.595033, 60.704758], [100.595003, 60.704776], [100.594972, 60.704795], [100.594942, 60.704814], [100.594911, 60.704833], [100.59488, 60.704852], [100.59485, 60.704871], [100.594819, 60.70489], [100.594789, 60.704909], [100.594758, 60.704928], [100.594728, 60.704946], [100.594697, 60.704965], [100.594667, 60.704984], [100.594636, 60.705003], [100.594606, 60.705022], [100.594575, 60.705041], [100.594545, 60.70506], [100.594514, 60.705079], [100.594484, 60.705098], [100.594453, 60.705116], [100.594422, 60.705135], [100.594392, 60.705154], [100.594361, 60.705173], [100.594331, 60.705192], [100.5943, 60.705211], [100.59427, 60.70523], [100.594239, 60.705249]]]]}, "id": "193868db-9e2d-4bcb-a167-8

In [32]:
asdf = check_orders()

In [25]:
asdf["items"][del_id]["deliveries"][0].keys()

dict_keys(['_links', 'id', 'kind', 'service', 'createdAt', 'status', 'isAmountEstimated', 'price', 'amount', 'estimatedAmount', 'amountUnit', 'area', 'unit', 'datas', 'updatedAt', 'deliveredAt'])

In [62]:
del_id = 0

In [67]:
links = asdf["items"][del_id]["deliveries"][0]["_links"]

In [68]:
radio = asdf["items"][del_id]["deliveries"][0]["datas"]["radiometricProcessing"]
image = asdf["items"][del_id]["deliveries"][0]["datas"]["imageFormat"]
processing = asdf["items"][del_id]["deliveries"][0]["datas"]["processingLevel"]

if "jp2" in image.lower():
    image = "jp2"
elif "geotiff" in image.lower():
    image = "geotiff"
else:
    image = "unknown"

fstring = f"{radio}-{processing}-{image}"

In [71]:
download(links["download"]["href"], f"output/{fstring}.zip")

502


In [72]:
links

{'self': {'href': 'https://data.api.oneatlas.airbus.com/api/v1/deliveries/cd037d41-5e76-42fb-9a06-26aade4ace17'},
 'contract': {'href': 'https://data.api.oneatlas.airbus.com/api/v1/contracts/cd55f842-5552-432b-b833-d29d00ca1643'},
 'catalogUrl': {'href': 'https://search.foundation.api.oneatlas.airbus.com/api/v1/opensearch?id=70ab339c-c22f-48d1-8870-7c2a207f876e'},
 'catalogItem': {'href': 'https://search.foundation.api.oneatlas.airbus.com/api/v1/opensearch?id=70ab339c-c22f-48d1-8870-7c2a207f876e'},
 'download': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/70ab339c-c22f-48d1-8870-7c2a207f876e/download'},
 'payment': {'href': 'https://data.api.oneatlas.airbus.com/api/v1/contracts/cd55f842-5552-432b-b833-d29d00ca1643/payments/f59bfe58-ec40-444f-9e80-64702dcc16ea'}}

In [125]:
asdfg = retrieve("193868db-9e2d-4bcb-a167-8bab1be09420")

In [128]:
asdfg["items"][0]["deliveries"][0]["_links"]

{'self': {'href': 'https://data.api.oneatlas.airbus.com/api/v1/deliveries/3c6ba88c-9fa2-4e15-9a19-75fb26f0c4c0'},
 'contract': {'href': 'https://data.api.oneatlas.airbus.com/api/v1/contracts/cd55f842-5552-432b-b833-d29d00ca1643'},
 'catalogUrl': {'href': 'https://search.foundation.api.oneatlas.airbus.com/api/v1/opensearch?id=0f57fe19-46ad-4cbc-b0bc-64329bead6a0'},
 'catalogItem': {'href': 'https://search.foundation.api.oneatlas.airbus.com/api/v1/opensearch?id=0f57fe19-46ad-4cbc-b0bc-64329bead6a0'},
 'download': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/0f57fe19-46ad-4cbc-b0bc-64329bead6a0/download'},
 'payment': {'href': 'https://data.api.oneatlas.airbus.com/api/v1/contracts/cd55f842-5552-432b-b833-d29d00ca1643/payments/12687a62-a5fb-48ba-b1f9-dfa68d5b6d0d'}}

In [64]:
img_id

'1bc481f3-4b64-4fd2-8cf3-3843928abb50'

In [22]:
simplejson.dumps(simplejson.dumps(payload))

'"{\\"kind\\": \\"order.data.gb.product\\", \\"products\\": [{\\"crsCode\\": \\"urn:ogc:def:crs:EPSG::4326\\", \\"productType\\": \\"pansharpened\\", \\"radiometricProcessing\\": \\"DISPLAY\\", \\"aoi\\": {\\"type\\": \\"Polygon\\", \\"coordinates\\": [[[1.4089965820312502, 43.59829500262717], [1.4089965820312502, 43.63408731864001], [1.5078735351562496, 43.63408731864001], [1.5078735351562496, 43.59829500262717], [1.4089965820312502, 43.59829500262717]]]}, \\"id\\": \\"3f33d1b8-ec54-433b-9808-107cee523637\\", \\"imageFormat\\": \\"image/jp2\\"}]}"'

In [63]:
auth_token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIxMThSRENzZTlqUWR4UVRnSkt2ZXlvSHBaaWE4R0pIVEU5RjJPSmE1M3N3In0.eyJleHAiOjE2MTQwNzYxODUsImlhdCI6MTYxNDA2MTc4NSwianRpIjoiMjIxNjc3NDItNGE0NC00OTU3LWJiNjItMDVhNDFiY2JjNTVhIiwiaXNzIjoiaHR0cHM6Ly9hdXRoZW50aWNhdGUuZm91bmRhdGlvbi5hcGkub25lYXRsYXMuYWlyYnVzLmNvbS9hdXRoL3JlYWxtcy9JRFAiLCJhdWQiOiJJRFAiLCJzdWIiOiI1MzYxODFkNy0zMmRjLTQ1MDQtYjIxOC1mZjg2OWJhNGEwYjIiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJJRFAiLCJzZXNzaW9uX3N0YXRlIjoiMmEwYjA0NWItNTEyMC00NDI0LWI2N2QtZDc0MDc2MTJmZWFiIiwiYWNyIjoiMSIsInNjb3BlIjoiIiwibmJmIjowLCJyb2xlIjoie1wiZ2VvLmFwcC5vYWRcIjpbXCJ1c2VyXCJdLFwiZ2VvLmlkcC5ub3RpZnlcIjpbXCJ1c2VyXCJdLFwiZ2VvLmlkcC5kYXRhc3RvcmVcIjpbXCJ1c2VyXCJdfSIsInJvbGVzIjp7Imdlby5hcHAub2FkIjpbInVzZXIiXSwiZ2VvLmlkcC5ub3RpZnkiOlsidXNlciJdLCJnZW8uaWRwLmRhdGFzdG9yZSI6WyJ1c2VyIl19LCJzdWlkIjoiMTE0MTc0NDA3MSIsInV1aWQiOiI4Mjc3YjEwNi05NzhhLTRhYmYtYWNmZi0wNTJjMTdkYTk3NGIiLCJsb2EiOjEwMH0.K5URWey7u37aMILPbcx2-r-p5sfb-iOKqReMC-38xjRZ4XSrcVM33sTDeE5A5JN81aif0L2XRRK9bBcKYu0dkT5mVie-gGQ

In [17]:
json.dump(payload)

TypeError: dump() missing 1 required positional argument: 'fp'

In [53]:
img_id = data["features"][3]["properties"]["id"]

img_id

'1bc481f3-4b64-4fd2-8cf3-3843928abb50'

In [45]:
data["features"][3]

{'_links': {'delete': {'href': 'https://search.foundation.api.oneatlas.airbus.com/api/v1/items/1bc481f3-4b64-4fd2-8cf3-3843928abb50',
   'name': 'Delete',
   'type': 'HTTP'},
  'imagesGetBuffer': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/1bc481f3-4b64-4fd2-8cf3-3843928abb50/images/50c6073e-438a-471b-b844-2a146bc6412f/buffer',
   'resourceId': '50c6073e-438a-471b-b844-2a146bc6412f',
   'type': 'getBuffer'},
  'imagesMetadata': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/1bc481f3-4b64-4fd2-8cf3-3843928abb50/images/50c6073e-438a-471b-b844-2a146bc6412f/metadata',
   'resourceId': '50c6073e-438a-471b-b844-2a146bc6412f',
   'type': 'application/geo+json'},
  'imagesWcs': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/1bc481f3-4b64-4fd2-8cf3-3843928abb50/images/50c6073e-438a-471b-b844-2a146bc6412f/wcs',
   'resourceId': '50c6073e-438a-471b-b844-2a146bc6412f',
   'type': 'WCS'},
  'imagesWms': {'href': 'https

In [60]:
polys[1]["geometry"]

{"coordinates": [[[[106.444089, 63.438518], [106.444125, 63.438538], [106.444161, 63.438557], [106.444196, 63.438577], [106.444232, 63.438597], [106.444268, 63.438617], [106.444304, 63.438637], [106.44434, 63.438657], [106.444375, 63.438677], [106.444411, 63.438697], [106.444447, 63.438717], [106.444483, 63.438737], [106.444519, 63.438757], [106.444554, 63.438777], [106.44459, 63.438797], [106.444626, 63.438817], [106.444662, 63.438837], [106.444698, 63.438856], [106.444733, 63.438876], [106.444769, 63.438896], [106.444805, 63.438916], [106.444841, 63.438936], [106.444877, 63.438956], [106.444913, 63.438976], [106.444948, 63.438996], [106.444984, 63.439016], [106.44502, 63.439036], [106.445056, 63.439056], [106.445091, 63.439076], [106.445127, 63.439096], [106.445163, 63.439116], [106.445199, 63.439135], [106.445235, 63.439155], [106.445271, 63.439175], [106.445306, 63.439195], [106.445342, 63.439215], [106.445378, 63.439235], [106.445414, 63.439255], [106.44545, 63.439275], [106.44548